In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
masters = pd.read_csv("../data-version2/raw/master-programs/master-programs.csv")

In [3]:
masters.head()

,field,name,university,duration,url,language,city,country,mode,deadline,pace,tution_amount,tution_currency
0,Administration-Studies,M.S. in Aviation Management And Logistics,Kent State University - College of Aeronautics...,2 years,NaN,"[""English""]",Kent,USA,"[""Online""]",NaN,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration-Studies,Master of Science in Operations Management,Manderson Graduate School of Business The Univ...,NaN,NaN,[],Tuscaloosa,USA,"[""Campus""]",NaN,"[""Full-time""]",NaN,NaN
2,Administration-Studies,MASTER'S DEGREE IN COLLECTIVE LABOR BARGAINING,University of Argentinian Social Studies,NaN,NaN,"[""Spanish (Argentina)""]",545,Argentina,"[""Campus""]",NaN,[],NaN,NaN
3,Administration-Studies,"MS Supply Chain, Logistics and Innovations",EM Normandie Business School,15 months,NaN,"[""French""]",Le Havre,France,"[""Campus""]",NaN,"[""Full-time""]",12500.0,EUR
4,Administration-Studies,Master of Logistics Management (without thesis),Izmir University of Economics,2 years,NaN,"[""English""]",Izmir,Turkey,"[""Campus""]",NaN,"[""Full-time""]",8500.0,USD


In [4]:
masters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23367 entries, 0 to 23366
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   field            23367 non-null  object 
 1   name             23367 non-null  object 
 2   university       23367 non-null  object 
 3   duration         17166 non-null  object 
 4   url              2197 non-null   object 
 5   language         23367 non-null  object 
 6   city             23366 non-null  object 
 7   country          23361 non-null  object 
 8   mode             23367 non-null  object 
 9   deadline         2975 non-null   object 
 10  pace             23367 non-null  object 
 11  tution_amount    13204 non-null  float64
 12  tution_currency  13154 non-null  object 
dtypes: float64(1), object(12)
memory usage: 2.3+ MB


In [5]:
# masters program table will not contain university name, city and country because they will be in a separate table

column_names : dict = {
    
    "pace"  : "schedule",
    "field" : "field_of_study",
    
}

columns_to_drop = ["university", "city", "country"]

masters_clean = masters.drop(columns_to_drop, axis=1)
masters_clean.rename(column_names, axis=1, inplace=True)

In [6]:
masters_clean.head()

,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration-Studies,M.S. in Aviation Management And Logistics,2 years,NaN,"[""English""]","[""Online""]",NaN,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration-Studies,Master of Science in Operations Management,NaN,NaN,[],"[""Campus""]",NaN,"[""Full-time""]",NaN,NaN
2,Administration-Studies,MASTER'S DEGREE IN COLLECTIVE LABOR BARGAINING,NaN,NaN,"[""Spanish (Argentina)""]","[""Campus""]",NaN,[],NaN,NaN
3,Administration-Studies,"MS Supply Chain, Logistics and Innovations",15 months,NaN,"[""French""]","[""Campus""]",NaN,"[""Full-time""]",12500.0,EUR
4,Administration-Studies,Master of Logistics Management (without thesis),2 years,NaN,"[""English""]","[""Campus""]",NaN,"[""Full-time""]",8500.0,USD


In [7]:

# convert NaN to NULL
masters_clean = masters_clean.fillna("NULL")


In [8]:

# for master programs table we need
# id, name, university_id, duration, language, mode, schedule, deadline, field_of_study, url, tution_amount, tution_currency

masters_clean.head()


,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration-Studies,M.S. in Aviation Management And Logistics,2 years,NULL,"[""English""]","[""Online""]",NULL,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration-Studies,Master of Science in Operations Management,NULL,NULL,[],"[""Campus""]",NULL,"[""Full-time""]",NULL,NULL
2,Administration-Studies,MASTER'S DEGREE IN COLLECTIVE LABOR BARGAINING,NULL,NULL,"[""Spanish (Argentina)""]","[""Campus""]",NULL,[],NULL,NULL
3,Administration-Studies,"MS Supply Chain, Logistics and Innovations",15 months,NULL,"[""French""]","[""Campus""]",NULL,"[""Full-time""]",12500.0,EUR
4,Administration-Studies,Master of Logistics Management (without thesis),2 years,NULL,"[""English""]","[""Campus""]",NULL,"[""Full-time""]",8500.0,USD


In [9]:

"""

To clean master program table:
    
    0. Clean field of studies
    1. Clean programs names
    2. Clean duration names
    3. Clean url
    4. Clean language, duplicate rows for multilanguage programs -> required for first norm of any database
    5. Duplicate multimode programs
    6. Clean dealine
    7. Duplicate multischedule programs
    8. Clean tution amount -> must be valid numbers
    9. Clean tution currency -> must be valid currency

"""


'\n\nTo clean master program table:\n    \n    0. Clean field of studies\n    1. Clean programs names\n    2. Clean duration names\n    3. Clean url\n    4. Clean language, duplicate rows for multilanguage programs -> required for first norm of any database\n    5. Duplicate multimode programs\n    6. Clean dealine\n    7. Duplicate multischedule programs\n    8. Clean tution amount -> must be valid numbers\n    9. Clean tution currency -> must be valid currency\n\n'

In [10]:
# field of study

fields = list(masters_clean["field_of_study"].value_counts().keys())
print(fields)


['Technology-Studies', 'Engineering-Studies', 'Management-Studies', 'Social-Sciences', 'Business-Studies', 'Economic-Studies', 'Humanities-Studies', 'Education', 'Natural-Sciences', 'Health-Care', 'Journalism-and-Mass-Communication', 'Administration-Studies', 'Life-Sciences', 'Marketing-Studies', 'Design-Studies', 'Art-Studies', 'Law-Studies', 'Environmental-Studies', 'Architecture-Studies', 'Sustainability-Studies', 'Energy-Studies', 'Performing-Arts', 'Tourism-and-Hospitality', 'Sport', 'Fashion', 'Languages', 'Construction', 'Professional-Studies', 'Food-and-Beverage-Studies', 'General-Studies', 'Aviation', 'Self-Improvement', 'Life-Skills', 'Cosmetology-Studies']


In [11]:

# we can remove -studies and '-' between words

fields_rename = {field : "".join(field.replace("-Studies", "").replace("Sciences", "").replace("and", "&").split("-")) for field in fields}

for key in fields_rename.keys():
    print(f"{key} => {fields_rename[key]}")


Technology-Studies => Technology
Engineering-Studies => Engineering
Management-Studies => Management
Social-Sciences => Social
Business-Studies => Business
Economic-Studies => Economic
Humanities-Studies => Humanities
Education => Education
Natural-Sciences => Natural
Health-Care => HealthCare
Journalism-and-Mass-Communication => Journalism&MassCommunication
Administration-Studies => Administration
Life-Sciences => Life
Marketing-Studies => Marketing
Design-Studies => Design
Art-Studies => Art
Law-Studies => Law
Environmental-Studies => Environmental
Architecture-Studies => Architecture
Sustainability-Studies => Sustainability
Energy-Studies => Energy
Performing-Arts => PerformingArts
Tourism-and-Hospitality => Tourism&Hospitality
Sport => Sport
Fashion => Fashion
Languages => Languages
Construction => Construction
Professional-Studies => Professional
Food-and-Beverage-Studies => Food&Beverage
General-Studies => General
Aviation => Aviation
Self-Improvement => SelfImprovement
Life-Skil

In [12]:

masters_clean = masters_clean.replace(fields_rename)
masters_clean.head()


,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration,M.S. in Aviation Management And Logistics,2 years,NULL,"[""English""]","[""Online""]",NULL,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration,Master of Science in Operations Management,NULL,NULL,[],"[""Campus""]",NULL,"[""Full-time""]",NULL,NULL
2,Administration,MASTER'S DEGREE IN COLLECTIVE LABOR BARGAINING,NULL,NULL,"[""Spanish (Argentina)""]","[""Campus""]",NULL,[],NULL,NULL
3,Administration,"MS Supply Chain, Logistics and Innovations",15 months,NULL,"[""French""]","[""Campus""]",NULL,"[""Full-time""]",12500.0,EUR
4,Administration,Master of Logistics Management (without thesis),2 years,NULL,"[""English""]","[""Campus""]",NULL,"[""Full-time""]",8500.0,USD


In [13]:
import re

def clean_name(name: str) -> bool:
    """
    
    Simple check names. A valid name should only contain A-Z characters or space.
    
    """
    
    clean_name = re.sub('[^a-zA-Z ]', '', name)
    
    return " ".join(clean_name.split()).title()

In [14]:
# clean name of programs

masters_clean["name"]    = masters_clean["name"].apply(clean_name)
masters_clean.head()


,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration,Ms In Aviation Management And Logistics,2 years,NULL,"[""English""]","[""Online""]",NULL,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration,Master Of Science In Operations Management,NULL,NULL,[],"[""Campus""]",NULL,"[""Full-time""]",NULL,NULL
2,Administration,Masters Degree In Collective Labor Bargaining,NULL,NULL,"[""Spanish (Argentina)""]","[""Campus""]",NULL,[],NULL,NULL
3,Administration,Ms Supply Chain Logistics And Innovations,15 months,NULL,"[""French""]","[""Campus""]",NULL,"[""Full-time""]",12500.0,EUR
4,Administration,Master Of Logistics Management Without Thesis,2 years,NULL,"[""English""]","[""Campus""]",NULL,"[""Full-time""]",8500.0,USD


In [15]:

# clean duration types
masters_clean["duration"].value_counts()


NULL                        6201
2 years                     5435
1 year                      2622
1&nbsp;-&nbsp;2 years       1396
12 months                    774
                            ... 
16&nbsp;-&nbsp;18 months       1
33 semesters                   1
1&nbsp;-&nbsp;12 years         1
24&nbsp;-&nbsp;42 years        1
136 hours                      1
Name: duration, Length: 374, dtype: int64

In [16]:

# need to remove &nbsp;
def clean_duration(duration: str) -> str:
    """
    
    Remove &nbsp; in duration.
    
    :param duration: duration of master program
    :return        : cleaned duration
    
    """
    
    if duration == "NULL":
        return duration
        
    return duration.replace("&nbsp;", "")

masters_clean["duration"] = masters_clean["duration"].apply(clean_duration)
masters_clean["duration"].value_counts()


NULL            6201
2 years         5435
1 year          2622
1-2 years       1396
12 months        774
                ... 
16-18 months       1
33 semesters       1
1-12 years         1
24-42 years        1
136 hours          1
Name: duration, Length: 374, dtype: int64

In [17]:

# clean url
masters_clean["url"].value_counts()


NULL                                                                                                                                                                                                      21170
http://www.kuleuven.be/onlineopendays                                                                                                                                                                       104
https://www.unibo.it/en/university/covid-19-The-measures-adopted-by-the-University-of-Bologna/covid-misure-universita-di-bologna/?utm_source=Keystone&utm_campaign=Keystone&utm_medium=KeystoneListing       94
https://www.lincoln.ac.uk/home/studywithus/internationalstudents/entryrequirementsandyourcountry/                                                                                                            71
https://bit.ly/3CLUQyS                                                                                                                                                  

In [18]:
# simple clean for url -> if url does not contain http set to NULL

from urllib.parse import urlparse

def is_url(url: str) -> bool:
    """
    
    Validate a url string.
    
    """
    try:
        result = urlparse(url)
    
        return all([result.scheme, result.netloc])
    
    except ValueError:
        return False

def clean_url(url: str) -> str:
    """
    
    Set url to NULL if url is not valid.
    
    :param url: program url
    :return   : valid url of NULL
    
    """
    
    if is_url(url):
        return url
    
    return "NULL"

masters_clean["url"] = masters_clean["url"].apply(clean_url)
masters_clean["url"].value_counts()


NULL                                                                                                                                                                                                      21226
http://www.kuleuven.be/onlineopendays                                                                                                                                                                       104
https://www.unibo.it/en/university/covid-19-The-measures-adopted-by-the-University-of-Bologna/covid-misure-universita-di-bologna/?utm_source=Keystone&utm_campaign=Keystone&utm_medium=KeystoneListing       94
https://www.lincoln.ac.uk/home/studywithus/internationalstudents/entryrequirementsandyourcountry/                                                                                                            71
https://bit.ly/3CLUQyS                                                                                                                                                  

In [19]:
masters_clean.head(50)

,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration,Ms In Aviation Management And Logistics,2 years,NULL,"[""English""]","[""Online""]",NULL,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration,Master Of Science In Operations Management,NULL,NULL,[],"[""Campus""]",NULL,"[""Full-time""]",NULL,NULL
2,Administration,Masters Degree In Collective Labor Bargaining,NULL,NULL,"[""Spanish (Argentina)""]","[""Campus""]",NULL,[],NULL,NULL
3,Administration,Ms Supply Chain Logistics And Innovations,15 months,NULL,"[""French""]","[""Campus""]",NULL,"[""Full-time""]",12500.0,EUR
4,Administration,Master Of Logistics Management Without Thesis,2 years,NULL,"[""English""]","[""Campus""]",NULL,"[""Full-time""]",8500.0,USD
5,Administration,Official Masters Degree In Human Resources Org...,9-24 months,NULL,"[""Spanish""]","[""Online""]",NULL,"[""Part-time""]",7560.0,EUR
6,Administration,Master In International Human Resources Manage...,2 years,NULL,"[""English""]","[""Online & Campus Combined"",""Campus"",""Online""]","""24 Dec 2021""","[""Full-time""]",222900.0,RUB
7,Administration,Management Of Public Organizations And Services,NULL,NULL,"[""Rumenian""]","[""Campus""]",NULL,[],NULL,NULL
8,Administration,Master Of Public Affairs,NULL,NULL,"[""English""]","[""Campus""]",NULL,[],NULL,NULL
9,Administration,Master Of Science In Human Resources Management,NULL,NULL,"[""English""]","[""Campus""]",NULL,[],NULL,NULL


In [20]:

# clean language
masters_clean["language"].value_counts()


["English"]                                16082
["Spanish"]                                 1673
[]                                          1107
["Russian"]                                  400
["Spanish (Mexican)"]                        377
                                           ...  
["Spanish","English","French","German"]        1
["English","Arabic","Malay"]                   1
["English","French","Spanish"]                 1
["English","Russian","Bulgarian"]              1
["English","Georgian"]                         1
Name: language, Length: 195, dtype: int64

In [21]:

# language column has multi values and this is not appropriate for first norm in database
# we need to repeat rows which have multi values
# database without first-norm is a bad database!


def clean_language(dataframe: pd.DataFrame) -> pd.DataFrame:
    
    colums   = dataframe.columns
    new_rows = []
    
    for row in tqdm(dataframe.values):
        
        for language in list(row[4].strip('[]').split(",")):
            
            new_row = row.copy()
            new_row[4] = language.strip('""')
            new_rows.append(new_row)
            
    return pd.DataFrame(data=new_rows, columns=colums)

    

In [22]:
masters_clean_language_1N = clean_language(masters_clean)

  0%|          | 0/23367 [00:00<?, ?it/s]

In [23]:
masters_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23367 entries, 0 to 23366
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   field_of_study   23367 non-null  object
 1   name             23367 non-null  object
 2   duration         23367 non-null  object
 3   url              23367 non-null  object
 4   language         23367 non-null  object
 5   mode             23367 non-null  object
 6   deadline         23367 non-null  object
 7   schedule         23367 non-null  object
 8   tution_amount    23367 non-null  object
 9   tution_currency  23367 non-null  object
dtypes: object(10)
memory usage: 1.8+ MB


In [24]:
masters_clean_language_1N.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25114 entries, 0 to 25113
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   field_of_study   25114 non-null  object
 1   name             25114 non-null  object
 2   duration         25114 non-null  object
 3   url              25114 non-null  object
 4   language         25114 non-null  object
 5   mode             25114 non-null  object
 6   deadline         25114 non-null  object
 7   schedule         25114 non-null  object
 8   tution_amount    25114 non-null  object
 9   tution_currency  25114 non-null  object
dtypes: object(10)
memory usage: 1.9+ MB


In [25]:
masters_clean_language_1N.head(50)

,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration,Ms In Aviation Management And Logistics,2 years,NULL,English,"[""Online""]",NULL,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration,Master Of Science In Operations Management,NULL,NULL,,"[""Campus""]",NULL,"[""Full-time""]",NULL,NULL
2,Administration,Masters Degree In Collective Labor Bargaining,NULL,NULL,Spanish (Argentina),"[""Campus""]",NULL,[],NULL,NULL
3,Administration,Ms Supply Chain Logistics And Innovations,15 months,NULL,French,"[""Campus""]",NULL,"[""Full-time""]",12500.0,EUR
4,Administration,Master Of Logistics Management Without Thesis,2 years,NULL,English,"[""Campus""]",NULL,"[""Full-time""]",8500.0,USD
5,Administration,Official Masters Degree In Human Resources Org...,9-24 months,NULL,Spanish,"[""Online""]",NULL,"[""Part-time""]",7560.0,EUR
6,Administration,Master In International Human Resources Manage...,2 years,NULL,English,"[""Online & Campus Combined"",""Campus"",""Online""]","""24 Dec 2021""","[""Full-time""]",222900.0,RUB
7,Administration,Management Of Public Organizations And Services,NULL,NULL,Rumenian,"[""Campus""]",NULL,[],NULL,NULL
8,Administration,Master Of Public Affairs,NULL,NULL,English,"[""Campus""]",NULL,[],NULL,NULL
9,Administration,Master Of Science In Human Resources Management,NULL,NULL,English,"[""Campus""]",NULL,[],NULL,NULL


In [26]:
# we can fill empty language values with NULL

masters_clean_language_1N.replace("", "NULL", inplace=True)


In [27]:

# apply first norm on mode values too
masters_clean_language_1N["mode"].value_counts()


["Campus"]                                        18207
["Online"]                                         3236
[]                                                 1697
["Online & Campus Combined","Campus","Online"]     1198
["Campus","Online"]                                 776
Name: mode, dtype: int64

In [28]:

def clean_mode(dataframe: pd.DataFrame) -> pd.DataFrame:
    
    colums   = dataframe.columns
    new_rows = []
    
    for row in tqdm(dataframe.values):
        
        for mode in list(row[5].strip('[]').split(",")):
            
            new_row = row.copy()
            new_row[5] = mode.strip('""')
            new_rows.append(new_row)
            
    return pd.DataFrame(data=new_rows, columns=colums)


In [29]:
masters_clean_language_mode_1N = clean_mode(masters_clean_language_1N)

  0%|          | 0/25114 [00:00<?, ?it/s]

In [30]:
masters_clean_language_1N.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25114 entries, 0 to 25113
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   field_of_study   25114 non-null  object
 1   name             25114 non-null  object
 2   duration         25114 non-null  object
 3   url              25114 non-null  object
 4   language         25114 non-null  object
 5   mode             25114 non-null  object
 6   deadline         25114 non-null  object
 7   schedule         25114 non-null  object
 8   tution_amount    25114 non-null  object
 9   tution_currency  25114 non-null  object
dtypes: object(10)
memory usage: 1.9+ MB


In [31]:
masters_clean_language_mode_1N.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28286 entries, 0 to 28285
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   field_of_study   28286 non-null  object
 1   name             28286 non-null  object
 2   duration         28286 non-null  object
 3   url              28286 non-null  object
 4   language         28286 non-null  object
 5   mode             28286 non-null  object
 6   deadline         28286 non-null  object
 7   schedule         28286 non-null  object
 8   tution_amount    28286 non-null  object
 9   tution_currency  28286 non-null  object
dtypes: object(10)
memory usage: 2.2+ MB


In [32]:
masters_clean_language_mode_1N["mode"].value_counts()

Campus                      20181
Online                       5210
                             1697
Online & Campus Combined     1198
Name: mode, dtype: int64

In [33]:
# we can fill empty language values with NULL
masters_clean_language_mode_1N.replace("", "NULL", inplace=True)

In [34]:
masters_clean_language_mode_1N["mode"].value_counts()

Campus                      20181
Online                       5210
NULL                         1697
Online & Campus Combined     1198
Name: mode, dtype: int64

In [35]:


# apply first norm on mode values too
masters_clean_language_mode_1N["deadline"].value_counts()


NULL             24452
"15 Feb 2022"      239
"17 Jan 2022"      233
"15 Jan 2022"      202
"30 Jun 2022"      185
                 ...  
"20 Feb 2022"        1
"02 May 2022"        1
"05 Aug 2022"        1
"31 Mar 2023"        1
"16 Apr 2022"        1
Name: deadline, Length: 136, dtype: int64

In [36]:

month_to_num = {
    
    "Jan" : "01",
    "Feb" : "02",
    "Mar" : "03",
    "Apr" : "04",
    "May" : "05",
    "Jun" : "06",
    "Jul" : "07",
    "Aug" : "08",
    "Sep" : "09",
    "Oct" : "10",
    "Nov" : "11",
    "Dec" : "12",
    
}

# we can convert deadline date to dd:mm:yyyy for postgres database
def clean_deadline(deadline: str) -> str:
        
    if deadline == "NULL":
        return deadline
    
    splits = deadline.split()
        
    return f"{splits[0]}:{month_to_num[splits[1]]}:{splits[2]}"
    
    

In [37]:
masters_clean_language_mode_1N["deadline"] = masters_clean_language_mode_1N["deadline"].apply(clean_deadline)

In [38]:
masters_clean_language_mode_1N["deadline"].value_counts()

NULL            24452
"15:02:2022"      239
"17:01:2022"      233
"15:01:2022"      202
"30:06:2022"      185
                ...  
"20:02:2022"        1
"02:05:2022"        1
"05:08:2022"        1
"31:03:2023"        1
"16:04:2022"        1
Name: deadline, Length: 136, dtype: int64

In [39]:
masters_clean_language_mode_1N.head()

,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration,Ms In Aviation Management And Logistics,2 years,NULL,English,Online,NULL,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration,Master Of Science In Operations Management,NULL,NULL,NULL,Campus,NULL,"[""Full-time""]",NULL,NULL
2,Administration,Masters Degree In Collective Labor Bargaining,NULL,NULL,Spanish (Argentina),Campus,NULL,[],NULL,NULL
3,Administration,Ms Supply Chain Logistics And Innovations,15 months,NULL,French,Campus,NULL,"[""Full-time""]",12500.0,EUR
4,Administration,Master Of Logistics Management Without Thesis,2 years,NULL,English,Campus,NULL,"[""Full-time""]",8500.0,USD


In [40]:

masters_clean_language_mode_1N["schedule"].value_counts()


["Full-time"]                13897
["Full-time","Part-time"]     6253
[]                            5880
["Part-time"]                 2256
Name: schedule, dtype: int64

In [41]:

# repeat rows for schedule too

def clean_schedule(dataframe: pd.DataFrame) -> pd.DataFrame:
    
    colums   = dataframe.columns
    new_rows = []
    
    for row in tqdm(dataframe.values):
        
        for schedule in list(row[7].strip('[]').split(",")):
            
            new_row = row.copy()
            new_row[7] = schedule.strip('""')
            new_rows.append(new_row)
            
    return pd.DataFrame(data=new_rows, columns=colums)


In [42]:
masters_clean_language_mode_schedule_1N = clean_schedule(masters_clean_language_mode_1N)

  0%|          | 0/28286 [00:00<?, ?it/s]

In [43]:
masters_clean_language_mode_schedule_1N.head()

,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration,Ms In Aviation Management And Logistics,2 years,NULL,English,Online,NULL,Full-time,42025.0,USD
1,Administration,Ms In Aviation Management And Logistics,2 years,NULL,English,Online,NULL,Part-time,42025.0,USD
2,Administration,Master Of Science In Operations Management,NULL,NULL,NULL,Campus,NULL,Full-time,NULL,NULL
3,Administration,Masters Degree In Collective Labor Bargaining,NULL,NULL,Spanish (Argentina),Campus,NULL,,NULL,NULL
4,Administration,Ms Supply Chain Logistics And Innovations,15 months,NULL,French,Campus,NULL,Full-time,12500.0,EUR


In [44]:
masters_clean_language_mode_schedule_1N.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34539 entries, 0 to 34538
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   field_of_study   34539 non-null  object
 1   name             34539 non-null  object
 2   duration         34539 non-null  object
 3   url              34539 non-null  object
 4   language         34539 non-null  object
 5   mode             34539 non-null  object
 6   deadline         34539 non-null  object
 7   schedule         34539 non-null  object
 8   tution_amount    34539 non-null  object
 9   tution_currency  34539 non-null  object
dtypes: object(10)
memory usage: 2.6+ MB


In [45]:
# we can fill empty language values with NULL
masters_clean_language_mode_schedule_1N.replace("", "NULL", inplace=True)

In [46]:

# cleaning process normally is done for the data 
# but we define enums in postgres database like below we need to make values same with enums

"""

enum Field {

  TECHNOLOGY
  ENGINEERING
  MANAGEMENT
  SOCIAL
  BUSINESS
  ECONOMIC
  HUMANITIES
  EDUCATION
  NATURAL
  HEALTHCARE
  JOURNALISM_AND_MASSCOMMUNICATION
  ADMINISTRATION
  LIFE
  MARKETING
  DESIGN
  ART
  LAW
  ENVIRONMENT
  ARCHITECTURE
  SUSTAINABILITY
  ENERGY
  PERFORMINGARTS
  TOURISM_AND_HOSTITALITY
  SPORT
  FASHION
  LANGUAGES
  CONSTRUCTION
  PROFESSIONAL
  FOOD_AND_BEVERAGE
  GENERAL
  AVIATION
  SELFIMPROVEMENT
  LIFESKILLS
  COSMETOLOGY

}

enum Schedule {
  FULLTIME
  PARTTIME
}

enum Mode {
  ONLINE
  CAMPUS
  ONLINE_AND_CAMPUS
}

"""


'\n\nenum Field {\n\n  TECHNOLOGY\n  ENGINEERING\n  MANAGEMENT\n  SOCIAL\n  BUSINESS\n  ECONOMIC\n  HUMANITIES\n  EDUCATION\n  NATURAL\n  HEALTHCARE\n  JOURNALISM_AND_MASSCOMMUNICATION\n  ADMINISTRATION\n  LIFE\n  MARKETING\n  DESIGN\n  ART\n  LAW\n  ENVIRONMENT\n  ARCHITECTURE\n  SUSTAINABILITY\n  ENERGY\n  PERFORMINGARTS\n  TOURISM_AND_HOSTITALITY\n  SPORT\n  FASHION\n  LANGUAGES\n  CONSTRUCTION\n  PROFESSIONAL\n  FOOD_AND_BEVERAGE\n  GENERAL\n  AVIATION\n  SELFIMPROVEMENT\n  LIFESKILLS\n  COSMETOLOGY\n\n}\n\nenum Schedule {\n  FULLTIME\n  PARTTIME\n}\n\nenum Mode {\n  ONLINE\n  CAMPUS\n  ONLINE_AND_CAMPUS\n}\n\n'

In [47]:
masters_clean_language_mode_schedule_1N["field_of_study"].value_counts()

Technology                      3376
Management                      2841
Social                          2721
Business                        2678
Engineering                     2494
Economic                        2271
Humanities                      1995
Education                       1840
Natural                         1494
HealthCare                      1346
Journalism&MassCommunication    1191
Marketing                       1051
Administration                  1046
Life                             960
Design                           818
Art                              791
Law                              787
Environmental                    683
Sustainability                   645
Architecture                     609
Energy                           480
Tourism&Hospitality              430
PerformingArts                   417
Sport                            326
Languages                        248
Fashion                          211
Construction                     186
P

In [48]:

def field_to_enum(field: str) -> str:
    return field.replace("&", "_AND_").upper()


In [49]:
masters_clean_language_mode_schedule_1N["field_of_study"] = masters_clean_language_mode_schedule_1N["field_of_study"].apply(field_to_enum)

In [50]:
masters_clean_language_mode_schedule_1N["field_of_study"].value_counts()

TECHNOLOGY                          3376
MANAGEMENT                          2841
SOCIAL                              2721
BUSINESS                            2678
ENGINEERING                         2494
ECONOMIC                            2271
HUMANITIES                          1995
EDUCATION                           1840
NATURAL                             1494
HEALTHCARE                          1346
JOURNALISM_AND_MASSCOMMUNICATION    1191
MARKETING                           1051
ADMINISTRATION                      1046
LIFE                                 960
DESIGN                               818
ART                                  791
LAW                                  787
ENVIRONMENTAL                        683
SUSTAINABILITY                       645
ARCHITECTURE                         609
ENERGY                               480
TOURISM_AND_HOSPITALITY              430
PERFORMINGARTS                       417
SPORT                                326
LANGUAGES       

In [51]:
masters_clean_language_mode_schedule_1N["mode"].value_counts()

Campus                      24134
Online                       7126
NULL                         1697
Online & Campus Combined     1582
Name: mode, dtype: int64

In [52]:

def mode_to_enum(mode: str) -> str:
    return mode.replace(" & ", "_AND_").upper()


In [53]:
masters_clean_language_mode_schedule_1N["mode"] = masters_clean_language_mode_schedule_1N["mode"].apply(mode_to_enum)

In [54]:
masters_clean_language_mode_schedule_1N["mode"].value_counts()

CAMPUS                        24134
ONLINE                         7126
NULL                           1697
ONLINE_AND_CAMPUS COMBINED     1582
Name: mode, dtype: int64

In [55]:
masters_clean_language_mode_schedule_1N["schedule"].value_counts()

Full-time    20150
Part-time     8509
NULL          5880
Name: schedule, dtype: int64

In [56]:

def schedule_to_enum(schedule: str) -> str:
    return schedule.replace("-", "").upper()


In [57]:
masters_clean_language_mode_schedule_1N["schedule"] = masters_clean_language_mode_schedule_1N["schedule"].apply(schedule_to_enum)

In [58]:
masters_clean_language_mode_schedule_1N["schedule"].value_counts()

FULLTIME    20150
PARTTIME     8509
NULL         5880
Name: schedule, dtype: int64

In [59]:
masters_clean_language_mode_schedule_1N.head()

,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,ADMINISTRATION,Ms In Aviation Management And Logistics,2 years,NULL,English,ONLINE,NULL,FULLTIME,42025.0,USD
1,ADMINISTRATION,Ms In Aviation Management And Logistics,2 years,NULL,English,ONLINE,NULL,PARTTIME,42025.0,USD
2,ADMINISTRATION,Master Of Science In Operations Management,NULL,NULL,NULL,CAMPUS,NULL,FULLTIME,NULL,NULL
3,ADMINISTRATION,Masters Degree In Collective Labor Bargaining,NULL,NULL,Spanish (Argentina),CAMPUS,NULL,NULL,NULL,NULL
4,ADMINISTRATION,Ms Supply Chain Logistics And Innovations,15 months,NULL,French,CAMPUS,NULL,FULLTIME,12500.0,EUR


In [60]:

cleaned_masters = masters_clean_language_mode_schedule_1N.copy()

# reset index delete old indices
cleaned_masters.reset_index(inplace=True)
cleaned_masters.drop(["index"], inplace=True, axis=1)

cleaned_masters.head()

# reset index and use as id
cleaned_masters.reset_index(inplace=True)
cleaned_masters = cleaned_masters.rename(columns={"index": "id"})


In [61]:
cleaned_masters.head()

,id,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,0,ADMINISTRATION,Ms In Aviation Management And Logistics,2 years,NULL,English,ONLINE,NULL,FULLTIME,42025.0,USD
1,1,ADMINISTRATION,Ms In Aviation Management And Logistics,2 years,NULL,English,ONLINE,NULL,PARTTIME,42025.0,USD
2,2,ADMINISTRATION,Master Of Science In Operations Management,NULL,NULL,NULL,CAMPUS,NULL,FULLTIME,NULL,NULL
3,3,ADMINISTRATION,Masters Degree In Collective Labor Bargaining,NULL,NULL,Spanish (Argentina),CAMPUS,NULL,NULL,NULL,NULL
4,4,ADMINISTRATION,Ms Supply Chain Logistics And Innovations,15 months,NULL,French,CAMPUS,NULL,FULLTIME,12500.0,EUR


In [62]:
cleaned_masters.to_csv("./clean/master_programs_clean.csv", index=False)

In [63]:

# check saved data
masters = pd.read_csv("./clean/master_programs_clean.csv")
masters.head()


,id,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,0,ADMINISTRATION,Ms In Aviation Management And Logistics,2 years,NaN,English,ONLINE,NaN,FULLTIME,42025.0,USD
1,1,ADMINISTRATION,Ms In Aviation Management And Logistics,2 years,NaN,English,ONLINE,NaN,PARTTIME,42025.0,USD
2,2,ADMINISTRATION,Master Of Science In Operations Management,NaN,NaN,NaN,CAMPUS,NaN,FULLTIME,NaN,NaN
3,3,ADMINISTRATION,Masters Degree In Collective Labor Bargaining,NaN,NaN,Spanish (Argentina),CAMPUS,NaN,NaN,NaN,NaN
4,4,ADMINISTRATION,Ms Supply Chain Logistics And Innovations,15 months,NaN,French,CAMPUS,NaN,FULLTIME,12500.0,EUR
